# New Section

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression


## **A.<U>Business constraints/requirements</U>:** <br> 1. No low latency requirements<br>2.Interpretability is very important<br>3.Very low error or no error<br>4.Probabilistic prediction for interpretablity

# <h1>B.Data Overview</h1>
<h2><h6>training_variants</h6>
<hr>
ID,Gene,Variation,Class<br>
0,FAM58A,Truncating Mutations,1 <br>
1,CBL,W802*,2 <br>
2,CBL,Q249E,2 <br>
...

<h6> training_text</h6>
<hr>
ID,Text <br>
0||Cyclin-dependent kinases (CDKs) regulate a variety of fundamental cellular processes. CDK10 stands out as one of the last orphan CDKs for which no activating cyclin has been identified and no kinase activity revealed. Previous work has shown that CDK10 silencing increases ETS2 (v-ets erythroblastosis virus E26 oncogene homolog 2)-driven activation of the MAPK pathway, which confers tamoxifen resistance to breast cancer cells. The precise mechanisms by which CDK10 modulates ETS2 activity, and more generally the functions of CDK10, remain elusive. Here we demonstrate that CDK10 is a cyclin-dependent kinase by identifying cyclin M as an activating cyclin. Cyclin M, an orphan cyclin, is the product of FAM58A, whose mutations cause STAR syndrome, a human developmental anomaly whose features include toe syndactyly, telecanthus, and anogenital and renal malformations. We show that STAR syndrome-associated cyclin M mutants are unable to interact with CDK10. Cyclin M silencing phenocopies CDK10 silencing in increasing c-Raf and in conferring tamoxifen resistance to breast cancer cells. CDK10/cyclin M phosphorylates ETS2 in vitro, and in cells it positively controls ETS2 degradation by the proteasome. ETS2 protein levels are increased in cells derived from a STAR patient, and this increase is attributable to decreased cyclin M levels. Altogether, our results reveal an additional regulatory mechanism for ETS2, which plays key roles in cancer and development. They also shed light on the molecular mechanisms underlying STAR syndrome.Cyclin-dependent kinases (CDKs) play a pivotal role in the control of a number of fundamental cellular processes (1). The human genome contains 21 genes encoding proteins that can be considered as members of the CDK family owing to their sequence similarity with bona fide CDKs, those known to be activated by cyclins (2). Although discovered almost 20 y ago (3, 4), CDK10 remains one of the two CDKs without an identified cyclin partner. This knowledge gap has largely impeded the exploration of its biological functions. CDK10 can act as a positive cell cycle regulator in some cells (5, 6) or as a tumor suppressor in others (7, 8). CDK10 interacts with the ETS2 (v-ets erythroblastosis virus E26 oncogene homolog 2) transcription factor and inhibits its transcriptional activity through an unknown mechanism (9). CDK10 knockdown derepresses ETS2, which increases the expression of the c-Raf protein kinase, activates the MAPK pathway, and induces resistance of MCF7 cells to tamoxifen (6). ... </h2>

# **C. Mapping to ML problem**
<br>1.Problem type: Multi class classification
<br>2.Metric: Multi Log-loss(9 classes)
<br>3.Metric: Multi class Confusion Matrix(With negligible FNR)-->shape=(9,9)

# **D. Exploratory Data Analysis**

In [ ]:
data_variants_tr = pd.read_csv('../input/cancer-data/training_variants')
print('shape of variants data:',data_variants_tr.shape)
data_variants_tr.head()

In [ ]:
# note the seprator in this file
data_text =pd.read_csv("../input/cancer-data/training_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
print('Number of data points : ', data_text.shape[0])
print('Number of features : ', data_text.shape[1])
print('Features : ', data_text.columns.values)
data_text.head()

In [ ]:
data_text['TEXT'][1]

#  **1.Text preprocessing: Here help has been taken from internet.**

In [ ]:
import nltk
nltk.download('stopwords')
# loading stop words from nltk library
stop_words = set(stopwords.words('english'))


def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        # replace every special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # replace multiple spaces with single space
        total_text = re.sub('\s+',' ', total_text)
        # converting all the chars into lower-case.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                string += word + " "
        
        data_text[column][index] = string

In [ ]:
data_text['TEXT']

In [ ]:
#text processing stage.
start_time = time.clock()
for index, row in data_text.iterrows():
    if type(row['TEXT']) is str:
        nlp_preprocessing(row['TEXT'], index, 'TEXT')
    else:
        print("there is no text description for id:",index)
print('Time took for preprocessing the text :',time.clock() - start_time, "seconds")

In [ ]:
result = pd.merge(data_variants_tr,data_text,on='ID',how='left')

In [ ]:
result.head()

In [ ]:
result.drop('ID',axis=1,inplace=True)

In [ ]:
result.shape

In [ ]:
result[result.isnull().any(axis=1)]

In [ ]:
#result = pd.merge(data_variants_tr,data_text,on='ID',how='left')
result.loc[result['TEXT'].isnull(),'TEXT'] = result['Gene'] +' '+result['Variation']
result[result['Gene']=='FANCA']

# **2.Splitting Data into train,test and CV**

In [ ]:

result.Gene      = result.Gene.str.replace('\s+', '_')
result.Variation = result.Variation.str.replace('\s+', '_')
result_x = result.drop('Class',axis=1)
result_y = result['Class']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(result,result_y,stratify=result_y,test_size=0.2)
X_tr,X_cv,Y_tr,Y_cv = train_test_split(X_train,Y_train,stratify=Y_train,test_size=0.2)

print('Train data shape: ',X_tr.shape)
print('Train class shape: ',Y_tr.shape)
print('Test data shape: ',X_test.shape)
print('Test class shape: ',Y_test.shape)
print('CV data shape: ',X_cv.shape)
print('CV class shape: ',Y_cv.shape)

In [ ]:
len(X_cv['Gene'].unique())

# **3.Checking for imbalance in data across train,test and CV**

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(Y_tr)
plt.title('Class distribution of train data',fontsize=30)
plt.xlabel('Class',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.show()
for i in range(1,10):
 print('share of class {0} is {1} with {2} datapoints.'.format(i,100*(Y_tr[Y_tr==i].count())/len(Y_tr),Y_tr[Y_tr==i].count()))

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(Y_test)
plt.title('Class distribution of test data',fontsize=30)
plt.xlabel('Class',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.show()
for i in range(1,10):
 print('share of class {0} is {1} with {2} datapoints.'.format(i,100*(Y_test[Y_test==i].count())/len(Y_test),Y_test[Y_test==i].count()))

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(Y_cv)
plt.title('Class distribution of CV data',fontsize=30)
plt.xlabel('Class',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.show()
for i in range(1,10):
 print('share of class {0} is {1} with {2} datapoints.'.format(i,100*(Y_cv[Y_cv==i].count())/len(Y_cv),Y_cv[Y_cv==i].count()))

# *Conclusion: <br>1.We have Imbalanced datasets among Class.<br>2. Our distribution of class across train, test and CV are almost same.*

# **4. Testing a dummy model and getting its log-loss score**<h5><br>Why we need dummy model?<br>Ans: Becuase the business requirement is to have multilog loss and we know that log-loss=[0,inf]. So if we have some log-loss let say 2.1 then we don't have reference to compare if this model is good or bad. So we need a dummy model  and compare our model with log-loss of dummy model to know where we are.

In [ ]:
# This function plots the confusion matrices given y_i, y_i_hat.
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)
    # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
    
    A =(((C.T)/(C.sum(axis=1))).T)
    #divid each element of the confusion matrix with the sum of elements in that column
    
    # C = [[1, 2],
    #     [3, 4]]
    # C.T = [[1, 3],
    #        [2, 4]]
    # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =1) = [[3, 7]]
    # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
    #                           [2/3, 4/7]]

    # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
    #                           [3/7, 4/7]]
    # sum of row elements = 1
    
    B =(C/C.sum(axis=0))
    #divid each element of the confusion matrix with the sum of elements in that row
    # C = [[1, 2],
    #     [3, 4]]
    # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =0) = [[4, 6]]
    # (C/C.sum(axis=0)) = [[1/4, 2/6],
    #                      [3/4, 4/6]] 
    
    labels = [1,2,3,4,5,6,7,8,9]
    # representing A in heatmap format
    print("-"*20, "Confusion matrix", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(C, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

    print("-"*20, "Precision matrix (Columm Sum=1)", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(B, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    
    # representing B in heatmap format
    print("-"*20, "Recall matrix (Row sum=1)", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(A, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()



    # we need to generate 9 numbers and the sum of numbers should be 1
# one solution is to genarate 9 numbers and divide each of the numbers by their sum
# ref: https://stackoverflow.com/a/18662466/4084039
test_data_len = X_test.shape[0]
cv_data_len = X_cv.shape[0]

# we create a output array that has exactly same size as the CV data
cv_predicted_y = np.zeros((cv_data_len,9))
for i in range(cv_data_len):
    rand_probs = np.random.rand(1,9)
    cv_predicted_y[i] = ((rand_probs/sum(sum(rand_probs)))[0])
print("Log loss on Cross Validation Data using Random Model",log_loss(Y_cv,cv_predicted_y, eps=1e-15))


# Test-Set error.
#we create a output array that has exactly same as the test data
test_predicted_y = np.zeros((test_data_len,9))
for i in range(test_data_len):
    rand_probs = np.random.rand(1,9)
    test_predicted_y[i] = ((rand_probs/sum(sum(rand_probs)))[0])
print("Log loss on Test Data using Random Model",log_loss(Y_test,test_predicted_y, eps=1e-15))

predicted_y =np.argmax(test_predicted_y, axis=1)
plot_confusion_matrix(Y_test, predicted_y+1)

# <I>Text Encoding block<br>1.Response Encoding with laplace smoothing<br>2. One Hot Encoding<br>3. TFIDF Encoding

In [ ]:
def res_encoding(cat_feature,tar_class):
  genes = list(set(cat_feature))
  vec_list = []
  #print(genes)
  df = pd.DataFrame({'cat_feature':cat_feature,'tar_class':tar_class})
  #print('line 2')
  #print(df)
  dic_feature_prob_given_target = {}
  for i in genes:
    dic_feature_prob_given_target[i]=[]
    for j in range(1,10):
         #print('inside 2nd for loop') 
         #print(df[(df.cat_feature==i) & (df.tar_class==j)].count() / (df[df.cat_feature==i].count()))
         dic_feature_prob_given_target[i].append(((df['cat_feature'][(df['cat_feature']==i) & (df['tar_class']==j)].count())+10) / ((df['cat_feature'][df['cat_feature']==i].count())+90))
         #print('below dict')
  for i in cat_feature:
    vec_list.append(dic_feature_prob_given_target[i])       
  return np.array(vec_list)




def one_hot_encoding(feature_tr,feature):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.preprocessing import StandardScaler
    scaler=StandardScaler()
    one_hot_encoder = CountVectorizer()
    one_hot_encoder.fit(feature_tr)
    one_hot_vec = one_hot_encoder.transform(feature).toarray()
    scaler.fit(one_hot_vec)
    return scaler.transform(one_hot_vec)


def tfidf_vectorizer(feature_tr,feature):
    from sklearn.feature_extraction.text import TfidfVectorizer 
    tfidf = TfidfVectorizer()
    tfidf.fit_transform(feature_tr)
    vec= tfidf.transform(feature).toarray()
    #print(tfidf.get_feature_names())
    return vec

# **5.Univariate Analysis**<br><h5>To check the feature importance when we have <b><I>less number of dimensions</I></b> in our original dataset is to build a simple sensible linear model on top of each feature and watch the performance of the model. If its performing considerably better than the dummy model score then this feature is important for our objective. Otherwise we have perform some other advanced testing on features.

# *5.1: Feature--> Gene*

# <h5>1. What kind of feature(data type) is this?<br>2.What are the different categories and how many are they?<br>3.How to featurize it?<br>4.How well this feature is performing?<br>5.6.Is this feature stable across all dataset(train,test and CV)?</h5><br><br>We will try to answer all these questions with good amount of reasoning.

# <h5>Ans(1): This feature is categorical type feature

In [ ]:
len(result['Gene'].unique())

# <h5>Ans(2): We have <b>262</b> different types of genes.<br>Ans(3): Response, One Hot, TFIDF

# *5.2:Vectorizing Gene*

In [ ]:
gene_one_hot_tr = one_hot_encoding(X_tr['Gene'],X_tr['Gene'])
gene_response_tr = res_encoding(X_tr['Gene'],Y_tr)
gene_tfidf_tr = tfidf_vectorizer(X_tr['Gene'],X_tr['Gene'])

gene_one_hot_cv = one_hot_encoding(X_tr['Gene'],X_cv['Gene'])
gene_response_cv = res_encoding(X_cv['Gene'],Y_cv)
gene_tfidf_cv = tfidf_vectorizer(X_tr['Gene'],X_cv['Gene'])

gene_one_hot_test = one_hot_encoding(X_tr['Gene'],X_test['Gene'])
gene_response_test = res_encoding(X_test['Gene'],Y_test)
gene_tfidf_test = tfidf_vectorizer(X_tr['Gene'],X_test['Gene'])

In [ ]:
gene_response_test.shape

# *5.3: Applying simple logistic regression on "Gene"  feature*

In [ ]:
uniq_val = result['Gene'].value_counts()
plt.figure(figsize=(15,10))
plt.plot(uniq_val.values / uniq_val.sum(),color='blue',linewidth=3)
plt.title('percentage share of genes',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of genes',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique genes= '+str(len(uniq_val)),xy=(180,0.075),fontsize=20,color='red' )
plt.show()


# *The curve drops very fast which means that very few of the gens are dominating in numbers.*

In [ ]:
cs = np.cumsum(uniq_val.values / uniq_val.sum())
plt.figure(figsize=(15,10))
plt.plot(cs,color='blue',linewidth=3)
plt.title('cummulative sum',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of genes',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique genes= '+str(len(uniq_val)),xy=(180,0.075),fontsize=20,color='red' )
plt.show()

# *We can see that only close to 60 genes comprise of 80% of the total genes. Or in other words we can say that 60/262 = 23 % of the total unique genes account for 80% of total genes.*

# *5.4: Checking performance of gene on LR model*

In [ ]:
def performance(vector_tr,vector_test,vector_cv,Y_tr,Y_test,Y_cv,alpha):
  error_cv=[]
  error_tr=[]
  error_test = []
  for i in alpha:
      clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
      clf.fit(vector_tr, Y_tr)
      sig_clf = CalibratedClassifierCV(clf, method="sigmoid") # we want our predicted value to be a probability for the interpretability hence we are using CalibratedClassifierCV
      sig_clf.fit(vector_tr,Y_tr)
      predict_y_tr = sig_clf.predict_proba(vector_tr)
      predict_y = sig_clf.predict_proba(vector_cv)
      predict_y_test = sig_clf.predict_proba(vector_test)
      error_cv.append(log_loss(Y_cv, predict_y, labels=clf.classes_, eps=1e-15))
      error_tr.append(log_loss(Y_tr, predict_y_tr, labels=clf.classes_, eps=1e-15))
      error_test.append(log_loss(Y_test, predict_y_test, labels=clf.classes_, eps=1e-15))  
  plt.figure(figsize=(15,10))
  plt.plot(error_cv,color='blue',linewidth=3)
  plt.plot(error_tr,color='grey',linewidth=3)
  plt.plot(error_test,color='red',linewidth=3)
  plt.title('performance checker',fontsize=30)
  plt.grid(b=True)
  plt.legend(['CV','Train','test'])
  plt.xlabel('Hyperparameter value',fontsize=20)
  plt.ylabel('log-loss',fontsize=20)
  #for i,j in zip(alpha,error_cv):
      #plt.annotate(str(round(j,2)),xy=(i,j),fontsize=20,color='grey' )
  plt.show()
  for i in (range(len(alpha))):
    print('Log loss is train =  {0}, test = {1} and cv = {2} for alpha value {3}'.format(error_tr[i],error_test[i],error_cv[i],alpha[i] )) 
 




In [ ]:
print('*'*50,'One Hot Encoding','*'*50)
alpha = [10 ** x for x in range(-5, 1)]
performance(gene_one_hot_tr,gene_one_hot_test,gene_one_hot_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'Response Encoding','*'*50)
performance(gene_response_tr,gene_response_test,gene_response_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'TFIDF Encoding','*'*50)
performance(gene_tfidf_tr,gene_tfidf_test,gene_tfidf_cv,Y_tr,Y_test,Y_cv,alpha) 

# ***Conclusions: As we can see the plots One Hot Encoding with Logistic Regression ic clear winner here. which gives <br>Log loss:<br>train =  0.9852677424374561 <br>test = 1.232340278209871 <br>cv = 1.1964023839802274 <br>for alpha value 0.01***<br>From this we van conclude that feature GENE is a very important feature when compared to dummy model which had logloss=2.5.

# **6: Feature: Variation**<br> we will do similar analysis  for this feature too as done for Gene.

# *6.1: Vectorization*

In [ ]:
var_one_hot_tr = one_hot_encoding(X_tr['Variation'],X_tr['Variation'])
var_response_tr = res_encoding(X_tr['Variation'],Y_tr)
var_tfidf_tr = tfidf_vectorizer(X_tr['Variation'],X_tr['Variation'])

var_one_hot_cv = one_hot_encoding(X_tr['Variation'],X_cv['Variation'])
var_response_cv = res_encoding(X_cv['Variation'],Y_cv)
var_tfidf_cv = tfidf_vectorizer(X_tr['Variation'],X_cv['Variation'])

var_one_hot_test = one_hot_encoding(X_tr['Variation'],X_test['Variation'])
var_response_test = res_encoding(X_test['Variation'],Y_test)
var_tfidf_test = tfidf_vectorizer(X_tr['Variation'],X_test['Variation'])

In [ ]:
uniq_val_var = result['Variation'].value_counts()
plt.figure(figsize=(15,10))
plt.plot(uniq_val_var.values / uniq_val_var.sum(),color='blue',linewidth=3)
plt.title('percentage share of Variation',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of Variation',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique variation= '+str(len(uniq_val_var)),xy=(2000,0.025),fontsize=20,color='red' )
plt.show()

In [ ]:
cs = np.cumsum(uniq_val_var.values / uniq_val_var.sum())
plt.figure(figsize=(15,10))
plt.plot(cs,color='blue',linewidth=3)
plt.title('cummulative sum',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of variation',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique variation= '+str(len(uniq_val_var)),xy=(180,0.075),fontsize=20,color='red' )
plt.show()

In [ ]:
print('*'*50,'One Hot Encoding','*'*50)
alpha = [10 ** x for x in range(-5, 1)]
performance(var_one_hot_tr,var_one_hot_test,var_one_hot_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'Response Encoding','*'*50)
performance(var_response_tr,var_response_test,var_response_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'TFIDF Encoding','*'*50)
performance(var_tfidf_tr,var_tfidf_test,var_tfidf_cv,Y_tr,Y_test,Y_cv,alpha) 

# *Conclusion: Best Parameters <br>LOG LOSS:<br>1.Train: Log loss is train = 0.5366492240360342, <br>2.test = 0.5413111923895613 <br>3.cv =  0.5427505443070724 <br>4alpha = 0.01*<br>5. Vectorizer: Response Encoding<br> Variation feature is not that strongly relatedto target value than GENE was. And we can also see that train loss and cv_loss and test_loss are having huge variance in terms of tfidf and one hot encoding. This tells us that this feature is trying to overfit the data. <br> But nonetheless it is performing way better than the dummy model and GENE feature. So we should definitely keep this feature. 

# *7: Feature: TEXT*

In [ ]:
text_one_hot_tr = one_hot_encoding(X_tr['TEXT'],X_tr['TEXT'])
text_response_tr = res_encoding(X_tr['TEXT'],Y_tr)
text_tfidf_tr = tfidf_vectorizer(X_tr['TEXT'],X_tr['TEXT'])

text_one_hot_cv = one_hot_encoding(X_tr['TEXT'],X_cv['TEXT'])
text_response_cv = res_encoding(X_cv['TEXT'],Y_cv)
text_tfidf_cv = tfidf_vectorizer(X_tr['TEXT'],X_cv['TEXT'])

text_one_hot_test = one_hot_encoding(X_tr['TEXT'],X_test['TEXT'])
text_response_test = res_encoding(X_test['TEXT'],Y_test)
text_tfidf_test = tfidf_vectorizer(X_tr['TEXT'],X_test['TEXT'])

In [ ]:
uniq_val_text = result['TEXT'].value_counts()
plt.figure(figsize=(15,10))
plt.plot(uniq_val_text.values / uniq_val_text.sum(),color='blue',linewidth=3)
plt.title('percentage share of TEXT',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of TEXT',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique TEXT= '+str(len(uniq_val_text)),xy=(1200,0.015),fontsize=20,color='red' )
plt.show()

cs = np.cumsum(uniq_val_text.values / uniq_val_text.sum())
plt.figure(figsize=(15,10))
plt.plot(cs,color='blue',linewidth=3)
plt.title('cummulative sum',fontsize=30)
plt.grid(b=True)
plt.xlabel('indices of TEXT',fontsize=20)
plt.ylabel('% share',fontsize=20)
plt.annotate('total unique TEXT= '+str(len(uniq_val_text  )),xy=(1200,0.015),fontsize=20,color='red' )
plt.show()

In [ ]:
print('*'*50,'One Hot Encoding','*'*50)
alpha = [10 ** x for x in range(-5, 1)]
performance(text_one_hot_tr,text_one_hot_test,text_one_hot_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'Response Encoding','*'*50)
performance(text_response_tr,text_response_test,text_response_cv,Y_tr,Y_test,Y_cv,alpha) 
print('*'*50,'TFIDF Encoding','*'*50)
performance(text_tfidf_tr,text_tfidf_test,text_tfidf_cv,Y_tr,Y_test,Y_cv,alpha) 

# *# *Conclusion: Best Parameters <br><h5>LOG LOSS:<br>1.Train: Log loss is train = 0.5883048303066178, <br>2.test = 0.6232510119525043 <br>3.cv = 0.631641928096624  <br>4alpha = 1e-05<br>5. Vectorizer: Response Encoding*<br> <h5>TEXT feature is  strongest related to target value than any other was. And we can also see that train loss and cv_loss and test_loss are having very less variance in terms of response encoding. This tells us that this feature is stable across datasets and maintains a good balance between underfitting and overfitting. <br>  So we should definitely keep this feature. *

# *Decision Making: <h5>So from above Univariate analysis of the features we will be doing:<br> 1.GENE: One Hot Encoding<br>2.Variation: Response Encoding<br>3. TEXT: Response Encoding*

# **D. Model Selection**
<h5> Since by now we know that how vectorization on each features has to be done.<br> But we don't know which model to apply. So we will try various model on our vectorized data.


*   Logistic Regression
*   Linear SVM
*   KNN
*   RandomForest

NOTE: As mentioned in the business requirements, we need interpretability of the model, hence we are notconsidering Ensembles as they don't have much of the interpretability. But Logistic and LinearSVM have high level of interpretability So we are testing these two models only. Just to compare we are taking RandomForest also.<br> And to make decision on choosing of models we will use confusion metrics, recall and precision metrics.





# *D.1: Data Preparation*

In [ ]:
data_tr = np.hstack((gene_one_hot_tr,var_response_tr,text_response_tr ))
data_test = np.hstack((gene_one_hot_test,var_response_test,text_response_test ))
data_cv = np.hstack((gene_one_hot_cv,var_response_cv,text_response_cv ))
print('train shape:',data_tr.shape)
print('test shape:',data_test.shape)
print('cv shape:',data_cv.shape)

# *D.2: Hyperparameter Tuning*

# *Logistic Regression*

In [ ]:
alpha = [10 ** x for x in range(-6, 3)] # this is the  lambda value we need to find out by cross validation 
def performance_model(vector_tr,vector_cv,Y_tr,Y_cv,alpha):
  error_cv=[]
  error_tr=[]
  for i in alpha:
      clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
      clf.fit(vector_tr, Y_tr)
      sig_clf = CalibratedClassifierCV(clf, method="sigmoid") # we want our predicted value to be a probability for the interpretability hence we are using CalibratedClassifierCV
      sig_clf.fit(vector_tr,Y_tr)
      predict_y_tr = sig_clf.predict_proba(vector_tr)
      predict_y = sig_clf.predict_proba(vector_cv)
      error_cv.append(log_loss(Y_cv, predict_y, labels=clf.classes_, eps=1e-15))
      error_tr.append(log_loss(Y_tr, predict_y_tr, labels=clf.classes_, eps=1e-15)) 
  plt.figure(figsize=(15,10))
  plt.plot(error_cv,color='blue',linewidth=3)
  plt.plot(error_tr,color='grey',linewidth=3)
  plt.title('performance checker',fontsize=30)
  plt.grid(b=True)
  plt.legend(['CV','Train'])
  plt.xlabel('Hyperparameter value',fontsize=20)
  plt.ylabel('log-loss',fontsize=20)
  #for i,j in zip(alpha,error_cv):
      #plt.annotate(str(round(j,2)),xy=(i,j),fontsize=20,color='grey' )
  plt.show()
  for i in (range(len(alpha))):
    print('Log loss is train =  {0} and cv = {1} for alpha value {2}'.format(error_tr[i],error_cv[i],alpha[i] )) 
 

In [ ]:
performance_model(data_tr,data_cv,Y_tr,Y_cv,alpha)

Best model = Log loss is train =  1.004839020557088 and cv = 1.2158399073823039 for alpha value 0.001

In [ ]:

# Now that we know our best model on whole dataset, we will train our model with best alpha value

def log_loss_and_confusion_matrix(data_tr, Y_tr,data_test, Y_test, model):
    model.fit(data_tr, Y_tr)
    sig_clf = CalibratedClassifierCV(model, method="sigmoid")
    sig_clf.fit(data_tr, Y_tr)
    pred_y = sig_clf.predict(data_test)

    # for calculating log_loss we willl provide the array of probabilities belongs to each class
    print("Log loss :",log_loss(Y_test, sig_clf.predict_proba(data_test)))
    # calculating the number of data points that are misclassified
    print("Number of mis-classified points :", np.count_nonzero((pred_y- Y_test))/Y_test.shape[0])
    plot_confusion_matrix(Y_test, pred_y)
    return pred_y 

In [ ]:
from sklearn.metrics import log_loss
model = SGDClassifier( class_weight='balanced',loss='log',penalty='l2',alpha=0.001)
log_loss_and_confusion_matrix(data_tr, Y_tr,data_test, Y_test, model)

# *LinearSVM*

In [ ]:
def performance_model_hinge(vector_tr,vector_cv,Y_tr,Y_cv,alpha):
  error_cv=[]
  error_tr=[]
  for i in alpha:
      clf = SGDClassifier(alpha=i, penalty='l2', loss='hinge', random_state=42)
      clf.fit(vector_tr, Y_tr)
      sig_clf = CalibratedClassifierCV(clf, method="sigmoid") # we want our predicted value to be a probability for the interpretability hence we are using CalibratedClassifierCV
      sig_clf.fit(vector_tr,Y_tr)
      predict_y_tr = sig_clf.predict_proba(vector_tr)
      predict_y = sig_clf.predict_proba(vector_cv)
      error_cv.append(log_loss(Y_cv, predict_y, labels=clf.classes_, eps=1e-15))
      error_tr.append(log_loss(Y_tr, predict_y_tr, labels=clf.classes_, eps=1e-15)) 
  plt.figure(figsize=(15,10))
  plt.plot(error_cv,color='blue',linewidth=3)
  plt.plot(error_tr,color='grey',linewidth=3)
  plt.title('performance checker',fontsize=30)
  plt.grid(b=True)
  plt.legend(['CV','Train'])
  plt.xlabel('Hyperparameter value',fontsize=20)
  plt.ylabel('log-loss',fontsize=20)
  #for i,j in zip(alpha,error_cv):
      #plt.annotate(str(round(j,2)),xy=(i,j),fontsize=20,color='grey' )
  plt.show()
  for i in (range(len(alpha))):
    print('Log loss is train =  {0} and cv = {1} for alpha value {2}'.format(error_tr[i],error_cv[i],alpha[i] )) 
 

In [ ]:
performance_model_hinge(data_tr,data_cv,Y_tr,Y_cv,alpha)

Best Model()

In [ ]:
model = SGDClassifier(loss='hinge',penalty='l2',alpha=100)
log_loss_and_confusion_matrix(data_tr, Y_tr,data_test, Y_test, model)

# *D.3: KNN*

In [ ]:
k = [1,3,5,7,9,11,13,15,17]
def performance_knn(vector_tr,vector_cv,Y_tr,Y_cv,alpha):
  from sklearn.neighbors import KNeighborsClassifier
  error_cv=[]
  error_tr=[]
  for i in alpha:
      clf = KNeighborsClassifier(n_neighbors=i)
      clf.fit(vector_tr, Y_tr)
      sig_clf = CalibratedClassifierCV(clf, method="sigmoid") # we want our predicted value to be a probability for the interpretability hence we are using CalibratedClassifierCV
      sig_clf.fit(vector_tr,Y_tr)
      predict_y_tr = sig_clf.predict_proba(vector_tr)
      predict_y = sig_clf.predict_proba(vector_cv)
      error_cv.append(log_loss(Y_cv, predict_y, labels=clf.classes_, eps=1e-15))
      error_tr.append(log_loss(Y_tr, predict_y_tr, labels=clf.classes_, eps=1e-15)) 
  plt.figure(figsize=(15,10))
  plt.plot(error_cv,color='blue',linewidth=3)
  plt.plot(error_tr,color='grey',linewidth=3)
  plt.title('performance checker',fontsize=30)
  plt.grid(b=True)
  plt.legend(['CV','Train'])
  plt.xlabel('Hyperparameter value',fontsize=20)
  plt.ylabel('log-loss',fontsize=20)
  #for i,j in zip(alpha,error_cv):
      #plt.annotate(str(round(j,2)),xy=(i,j),fontsize=20,color='grey' )
  plt.show()
  for i in (range(len(alpha))):
    print('Log loss is train =  {0} and cv = {1} for alpha value {2}'.format(error_tr[i],error_cv[i],alpha[i] )) 

In [ ]:
performance_knn(data_tr,data_cv,Y_tr,Y_cv,k)

In [ ]:
model = KNeighborsClassifier(n_neighbors=17)
log_loss_and_confusion_matrix(data_tr, Y_tr,data_test, Y_test, model)

# *Conclusions: Surprisingly KNN is performing alot better than LinearSVM and Logistic Regression. The difference of missclassification is almost 20 %. So for now our model is KNN with n_neighbors=17, Why?, because less than 17 we will overfit the data.*

# *D.4: RandomForest*

In [ ]:
def performance_rf(vector_tr,vector_cv,Y_tr,Y_cv,alpha):
  from sklearn.ensemble import RandomForestClassifier
  for l in [5,10]:
    error_cv=[]
    error_tr=[]
    for i in alpha:
        clf = RandomForestClassifier(n_estimators=i,criterion='gini',max_depth=l)
        clf.fit(vector_tr, Y_tr)
        sig_clf = CalibratedClassifierCV(clf, method="sigmoid") # we want our predicted value to be a probability for the interpretability hence we are using CalibratedClassifierCV
        sig_clf.fit(vector_tr,Y_tr)
        predict_y_tr = sig_clf.predict_proba(vector_tr)
        predict_y = sig_clf.predict_proba(vector_cv)
        error_cv.append(log_loss(Y_cv, predict_y, labels=clf.classes_, eps=1e-15))
        error_tr.append(log_loss(Y_tr, predict_y_tr, labels=clf.classes_, eps=1e-15)) 
    plt.figure(figsize=(15,10))
    plt.plot(error_cv,color='blue',linewidth=3)
    plt.plot(error_tr,color='grey',linewidth=3)
    plt.title('performance checker max_depth='+str(l),fontsize=30)
    plt.grid(b=True)
    plt.legend(['CV','Train'])
    plt.xlabel('Hyperparameter value',fontsize=20)
    plt.ylabel('log-loss',fontsize=20)
    #for i,j in zip(alpha,error_cv):
        #plt.annotate(str(round(j,2)),xy=(i,j),fontsize=20,color='grey' )
    plt.show()
    for i in (range(len(alpha))):
      print('Log loss is train =  {0} and cv = {1} for alpha value {2}'.format(error_tr[i],error_cv[i],alpha[i] )) 

In [ ]:
alpha = [100,200,500,1000,2000]
performance_rf(data_tr,data_cv,Y_tr,Y_cv,alpha)

# *New Champion: Log loss is train =  0.03967625782118917 and cv = 0.04036323580651017 for alpha value 200 and max_depth=5*

In [ ]:
model = RandomForestClassifier(n_estimators=200,criterion='gini',max_depth=5)
prediction_test=log_loss_and_confusion_matrix(data_tr, Y_tr,data_test, Y_test, model)

# *Conclusion: RandomForest is working like magical. But the problem with RandomForest is it is not interpretable. So we can't use this.*